In [55]:
import glob
import regex as re

date = re.compile("[<>]")

def date_repl(grp):
    if grp.group(0) == "<":
        return "<date>"
    elif grp.group(0) == ">":
        return "</date>"
    print(grp)
    return ""

books = {
    
}
for text in glob.glob("*.txt"):
    number = text.replace(".txt", "")
    print(number)
    with open(text) as f:
        data = f.read()
    data = date.sub(date_repl, data)
    data = re.sub(
        f"{number}\.(\d+)\.0\s*(.*)\s*\n", 
        '\n</p></div>\n</div>\n<div type="textpart" subtype="chapter" n="\g<1>">\n<head>\g<2></head>\n\n',
        data
    )
    data = re.sub(
        f"{number}\.\d+\.1\s+",
        '\n<div type="textpart" subtype="section" n="1">\n<p n="0">\n',
        data
    )
    data = re.sub(
        f"{number}\.\d+\.(\d+)\s+",
        '\n</p>\n</div>\n<div type="textpart" subtype="section" n="\g<1>">\n<p n="0">\n',
        data
    )
    data = re.sub(
        r"(\d+)\s*\.\s*",
        '\n</p>\n<p n="\g<1>">\n',
        data
    )
    data = re.sub(
        r"(<p n=\"0\">\s+)(Imp[^.]+\.)",
        '\n<head>\g<2></head>\n\g<1>',
        data
    )
    data = re.sub(
        r"(\*\s*<?)",
        lambda x: "</persName> <" if x.group(1)[-1] == "<" else "<persName>",
        data
    )
    data = data.replace("[Here there is a Greek text. Sorry, it is not yet in the Library.]", '<gap reason="omitted" />')
    data += "</p>\n</div>\n</div>\n</div>"
    # Replace the top stuff
    while data.startswith("\n") or data.startswith("</"):
        data = "\n".join(data.split("\n")[1:])
    data = f'<div type="textpart" subtype="book" n="{number}">\n' + data
    books[int(number)] = data
    with open(f"{number}.xml", "w") as f:
        f.write(data)

1
4
2
12
9
3
11
10
7
6
5
8


In [56]:
with open("final.xml", "w") as f:
    f.write("""<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Codex Iustinianus</title>
                <author>Iustinianus</author>
            </titleStmt>
            <publicationStmt>
                <p>This document has been adapted from the Corpus Corporum project for Capitains
                    formating.</p>
            </publicationStmt>
            <sourceDesc>
                <bibl>
                    <author ref="http://viaf.org/viaf/5725215">Gaius<date>fl.160</date></author>
                    <title>Institutiones</title>
                    <editor>E. Seckel &amp; B. Kuebler, Gai Institutionum commentarii quattuor... </editor>
                    <idno type="URI">http://droitromain.upmf-grenoble.fr/Responsa/gai1.htm</idno>
                    <pubPlace>Lipsiae</pubPlace>
                    <publisher>Teubner</publisher>
                    <date>1908</date>
                    <edition>critical edition, apparatus not encoded</edition>
                </bibl>
            </sourceDesc>
        </fileDesc>
        <encodingDesc>
            <projectDesc>
                <p>this file was encoded in TEI xml for the University of Zurich's <title>Corpus
                        Corporum</title> project (www.mlat.uzh.ch) by Ph. Roelli in 2013</p>
            </projectDesc>
            <refsDecl n="CTS">
                <cRefPattern n="section" matchPattern="(\w+).(\w+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n='$1']//tei:div[@subtype='section' and @n='$2'])">
                    <p>This pointer pattern extracts chapter</p>
                </cRefPattern>
                <cRefPattern n="book" matchPattern="(\w+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n='$1'])">
                    <p>This pointer pattern extracts books</p>
                </cRefPattern>
            </refsDecl>
        </encodingDesc>
        <revisionDesc>
            <change when="2021-04-02" who="Thibault Clérice">Added <![CDATA[<gap/>]]> and section for citability in Capitains</change>
        </revisionDesc>
    </teiHeader>
    <text>
        <body>
            <div type="edition" n="urn:cts:latinLit:stoa0168.stoa001c.corpuscorporum-lat1" xml:lang="lat">
                {}
            </div>
        </body>
    </text>
</TEI>
""".format("\n".join([
        books[numb]
        for numb in sorted(list(books.keys()))
    ])))